# RERANK
- 트랜스포머(버트) 크로스인코더 구조 활용해서 더욱 정밀한 시맨틱 유사도 스코어링 및 이에 기반한 문서 재정렬

In [ ]:
!pip install llama-index
!pip install llama-index-postprocessor-cohere-rerank
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found exist

In [ ]:
# set up OpenAI
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key:··········


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)
Settings.llm= OpenAI(model='gpt-4o-mini')

In [ ]:
# load documents
documents = SimpleDirectoryReader("/content/pg").load_data()

# build index
index = VectorStoreIndex.from_documents(documents=documents)

In [ ]:
import os
from llama_index.postprocessor.cohere_rerank import CohereRerank

#cohere api 키
api_key = ''

#리랭커 top_2로 정의
cohere_rerank = CohereRerank(api_key=api_key, top_n=2)

In [ ]:
# 어차피 리랭크로 재소팅할거니까 넉넉한 top_k값 부여
query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[cohere_rerank],
)
response = query_engine.query(
    "What did Sam Altman do in this essay?",
)

In [ ]:
pprint_response(response, show_source=True)

Final Response: Sam Altman was one of the founders in the first batch
of the Summer Founders Program, which was part of Y Combinator. He
later became the second president of Y Combinator.
______________________________________________________________________
Source Node 1/2
Node ID: 00357539-2337-4587-9f83-6ba3a101a2fb
Similarity: 0.74853617
Text: They were an impressive group. That first batch included reddit,
Justin Kan and Emmett Shear, who went on to found Twitch, Aaron
Swartz, who had already helped write the RSS spec and would a few
years later become a martyr for open access, and Sam Altman, who would
later become the second president of YC. I don't think it was entirely
luck that ...
______________________________________________________________________
Source Node 2/2
Node ID: cc1cadc0-6562-41a8-9a34-f6777d80d43f
Similarity: 0.70416015
Text: [13]  Once again, ignorance worked in our favor. We had no idea
how to be angel investors, and in Boston in 2005 there were no Ron
Conway

In [ ]:
# naive 쿼리 결과 비교
query_engine = index.as_query_engine(
    similarity_top_k=2,
)
response = query_engine.query(
    "What did Sam Altman do in this essay?",
)

In [ ]:
pprint_response(response, show_source=True)

Final Response: Sam Altman was recruited to become the president of Y
Combinator after the original founders decided to reorganize the
company for its long-term sustainability. Initially, he declined the
offer as he was interested in starting a startup focused on nuclear
reactors. However, after continued discussions, he agreed to take over
starting with the winter 2014 batch, allowing the original founders to
step back and transition control of Y Combinator to him.
______________________________________________________________________
Source Node 1/2
Node ID: 36e932f6-056c-4106-b2b2-78cacaf985a0
Similarity: 0.5203709271603304
Text: But while I continued to work a good deal in Arc, I gradually
stopped working on Arc, partly because I didn't have time to, and
partly because it was a lot less attractive to mess around with the
language now that we had all this infrastructure depending on it. So
now my three projects were reduced to two: writing essays and working
on YC.  YC w...
________

In [ ]:
# Dataset 로드
from datasets import load_dataset

ds = load_dataset("HAERAE-HUB/KOREAN-WEBTEXT", split='train[:20]')
data = ds.to_pandas()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1284879 [00:00<?, ? examples/s]

In [ ]:
# Document 오브젝트로 변환
from llama_index.core import Document, VectorStoreIndex
docs = []

#Iterative하게 Document 만들기
for i, row in data.iterrows():
    docs.append(Document(
        text=row['text'],
        # extra_info={'title': row['title']}
    ))

In [ ]:
index = VectorStoreIndex.from_documents(
    docs
)

In [ ]:
# naive engine
query_engine = index.as_query_engine(
    similarity_top_k=2
)

# rerank engine
rerank_query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[cohere_rerank],
)

In [ ]:
# naive Response
naive_response = query_engine.query(
    "분당과 같은 도시가 외국에서도 통하려면 어떻게해야되?",
)
#rerank response
rerank_response = rerank_query_engine.query(
    "분당과 같은 도시가 외국에서도 통하려면 어떻게해야되?",
)

In [ ]:
pprint_response(naive_response, show_source=True)
print('------------------------------RERANK-----------------------------')
pprint_response(rerank_response, show_source=True)

Final Response: 외국에서 분당과 같은 도시가 성공적으로 자리 잡기 위해서는 해당 지역의 사람, 문화, 시장,
관계, 제도 등을 깊이 이해하는 것이 중요합니다. 또한, 현지의 특성과 요구에 맞춘 주택 상품을 기획하고, 시행과 시공을
분리하여 경제 상황에 유연하게 대응할 수 있는 전략이 필요합니다. 이를 통해 로컬 시장에 적합한 접근 방식을 개발하고,
성공적인 커뮤니티 형성을 도모해야 합니다.
______________________________________________________________________
Source Node 1/2
Node ID: 771f70e8-9f38-4b88-b4ba-e56693231c62
Similarity: 0.27829180299373363
Text: 이러한 비정기적인 지출에 대비해 #2.예비비통장을 만들어 두어야 합니다. 이 예비비통장에는 매월 정기 수입의 10%
정도를 떼어서 입금해 두는 것이 좋습니다. 이 통장에 돈이 없으면 신용카드나 마이너스 대출, 카드론 등을 사용하게 되는데 이는
지출의 악순환이 반복되는 지름길입니다. 정기적인 지출은 재테크 통장과 생활비 통장으로 구분할 수 있습니다. #3. 재테크
통장은 매월 고정적으로 나가는 저축・투자 및 지출 항목을 자동이체로 설정해 두는 통장입니다. 재테크통장에서 이체되는 돈은
적금, 보험, 연금, 펀드, 대출원리금 등입니다. 이때 월급을 나누는 비율이 중요합니다. 적금, 펀드 등을 위한
저축・투자통장으로 15...
______________________________________________________________________
Source Node 2/2
Node ID: adc15553-b290-44d6-8c64-bba30b08ff24
Similarity: 0.2593159310041085
Text: 초기 분양상품은 한채 약 7만불 내외의 흥붕 및 스카이가든 아파트였고 분양은 매우 성공적이었다. 많은 사람들이
베트남에서 아파트문화를 새롭게 정의

In [ ]:
# naive Response
naive_response = query_engine.query(
    "프랭키가 누구한테 인기가 없을까?",
)
#rerank response
rerank_response = rerank_query_engine.query(
    "프랭키가 누구한테 인기가 없을까?",
)

In [ ]:
pprint_response(naive_response, show_source=True)
print('------------------------------RERANK-----------------------------')
pprint_response(rerank_response, show_source=True)

Final Response: 프랭키는 일본 축구에 대한 한국인의 비판적인 시각 속에서 인기가 없을 것으로 보인다. 특히, 한국
축구가 일본 축구에 비해 우위를 주장하는 경향과 관련하여, 일본 축구에 대한 사실과 다른 인식이 나타나고 있어, 이러한
맥락에서 프랭키의 인기도 영향을 받을 수 있다.
______________________________________________________________________
Source Node 1/2
Node ID: afc4734f-e58c-47ec-99d7-569586382d3f
Similarity: 0.21388324472309853
Text: 일기라는 가장 내밀한 기록 속에 드러나는 세계제국 영국 역사의 치부, 성이라는 가장 사적인 생활에 개입된 계급적
한계가 사회 최상류층이자 최약자로서 찰스와 윌을 통해 구현되면서 소설은 두터운 현실감을 얻는다. 작가 특유의 정교한 구성과
섬세하고 날렵한 문체에 읽는 재미를 더하는 것은 군데군데 숨은 풍자다. 영국 왕위계승 서열을 그대로 따른 주요 인물들의 이름,
저소득층을 상대로 자선을 베푸는 한편 그들을 꾀어 포르노 영화를 찍으며 감상하는 사진가의 이름에 오점, 얼룩(stain)의
뜻이 담긴 점, 아르헨띠나인과 영국의 포클랜드 침략에 관해 대화를 나누면서 윌이 거꾸로 희롱당하는 입장에 처하는 장면 등은
우스꽝스럽게...
______________________________________________________________________
Source Node 2/2
Node ID: 79b4a828-ec36-48de-8ebd-8a0e6163b6f3
Similarity: 0.20567095593572832
Text: 아마도 대한민국에서 일본보다 우리나라 성인축구 수준이 한참이나 떨어진다고 하면 이를 인정하는 사람은 거의 없다.
언제나 일본축구보다 우위에 있다는 정신승리자위질에 빠져 있다. 아시안컵에서 우리나라는 50년대 2번 우승한 것을 제외하고는
지금까지 단 한번도

In [ ]:
# naive Response
naive_response = query_engine.query(
    "동양 섬나라 국가랑 서양국가가 싸운 전쟁이 뭐였지?",
)
#rerank response
rerank_response = rerank_query_engine.query(
    "동양 섬나라 국가랑 서양국가가 싸운 전쟁이 뭐였지?",
)

In [ ]:
pprint_response(naive_response, show_source=True)
print('------------------------------RERANK-----------------------------')
pprint_response(rerank_response, show_source=True)

Final Response: 동양 섬나라 국가와 서양 국가 간의 전쟁에 대한 정보는 제공된 내용에 포함되어 있지 않습니다.
______________________________________________________________________
Source Node 1/2
Node ID: b24366e2-cbdd-4b71-ab9e-ef3464921f22
Similarity: 0.3067140142612102
Text: “아~아~잊으랴 어찌 우리 이날을 조국의 원수들이 짓밟아 오던 날을~”6·25의 노래 시작부이다. 1950년에
발발하여 남북한 인구 600만명 이상의 인명피해를 남기고 전국토의 80%이상을 파괴하였던 6·25 전쟁은 발발 3년 뒤인
1953년 7월에 휴전이 되어 지금에까지 이르고있다.1955년 6월은 6·25에 대한 기억이 생생할 때이다. 1955년 6월
25일에는 6·25 발발 다섯돌을 맞아 국민대회가 폭우를 무릎쓰고 진행되었다고 당시 일간지들은 보도하고 있다. 하나님께서는
1955년 6월 9일부터 16일까지 대구집회를 마치고 일주일 뒤인 6월 23일부터 30일까지 용산 군인교회에서 집회를
가지셨다. 신문에 나간...
______________________________________________________________________
Source Node 2/2
Node ID: 706d4146-0881-47f5-81f4-972baee2bc00
Similarity: 0.28411575397282385
Text: 교묘하게 반일감정을 자극하여 조센징 개돼지들을 선동하고 있다. 일본의 경제력은 한국과 프랑스의 경제력을 합쳐 놓은
것과 비슷하다. 한국의 경제력은 정확하게 프랑스 경제력의 절반이다. 앞으로 일본 경제력을 추월할 일은 벌어지지 않을 것이고,
남북통일이 되어도 일본의 경제력을 능가할 가능성은 없다. 게다가, 국민성 마져도 일본은 적에게서 배우고 실패에서 배운다.
이성적이고 공공의 이익을 추구할 줄 안다. 그에 비하여 한국은 이성

In [ ]:
# naive Response
naive_response = query_engine.query(
    "빚부터 갚으라는 유튜버가 누구지?",
)
#rerank response
rerank_response = rerank_query_engine.query(
    "빚부터 갚으라는 유튜버가 누구지?",
)

In [ ]:
pprint_response(naive_response, show_source=True)
print('------------------------------RERANK-----------------------------')
pprint_response(rerank_response, show_source=True)

Final Response: 오상열입니다.
______________________________________________________________________
Source Node 1/2
Node ID: 13392576-c3c7-4ad6-ac93-2ec32ad4cbb2
Similarity: 0.27023516430882044
Text: 거리와 색상을 통해 자신이 해야 할 일을 판단하는 거죠. 간단하게 1~2 단계 움직이는 모습을 영상으로 담아
봤는데요. 고양이 울음소리를 내고 감정을 표현하는 게 영락없는 고양이입니다. 맨날 싸우는 우리 아이들이지만 레고 부스트를
만드는 시간만큼은 세상 둘도 없는 오누이입니다. 둘째는 자기가 만들지 못할 것을 아니까 구경하면서 응원하고 첫째가 열심히
만들죠. 지난번에도 그러더니 꼭 친한 척을 하네요. 짠 ~ 다리까지 모두 완성된 프랭키입니다. 귀엽죠? 나름 리본도 매달고
있습니다. 울 따님은 너무 귀엽다며 계속 이방 저방 데리고 다니고 있습니다. 기다란 꼬리를 가지고 있는데, 꼬리까지 살랑살랑
흔들면서 진짜 고양이...
______________________________________________________________________
Source Node 2/2
Node ID: 2b408fdc-1b14-4210-976e-afc649d3ab88
Similarity: 0.2669967784732456
Text: 재테크 채널 유튜버이자, 「빚부터 갚아라」, 「원트재무설계 소원을 말해봐」 저자인 오상열의 <어차피 재테크는 토스>
칼럼 시리즈에서는, 재테크의 가장 기본인 통장 관리부터 채권 투자까지 전반적인 돈 관리 방법을 하나씩 짚게 됩니다. 모두가
경제적 자유를 얻을 수 있도록, 알찬 재테크 정보를 쉽고 재미있게 전달합니다. 월급이 들어오는 통장 관리, 어떻게 하고
계시나요? 월급이란 근로와 사업의 댓가로 매월 통장으로 들어오는 수입을 말합니다. 이 수입을 가지고 생활비로도 쓰고, 저축과
투자도 하고, 보험료도 